In [1]:
import pandas as pd
from geonamescache import GeonamesCache
import re
#from unidecode import unidecode
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

# 1. Obtener dataset

Empezamos cargando el archivo que contine las diferentes ofertas empleo. Estas ofertas se han obtenido de tres portales de empleo que demandan personas con competencias en Data Science y son los siguientes:
* [reed.co.uk](https://www.reed.co.uk/jobs/data-science-jobs)
* [itjobs.ca](https://www.itjobs.ca/en/search-jobs/?keywords=data+science&search=1)
* [jobscube](https://jobcube.com/Data-Science-jobs)

1. Cargar Data frame

In [58]:
df=pd.read_csv('jobs_titles.csv')
df.head()

,title,portal
0,Data Science Manager job in New York NY,Jobcube
1,Director Data Science job in Seattle WA,Jobcube
2,Data Science Instructor job in San Antonio TX,Jobcube
3,"Senior Manager, Sales Data Science and Analyti...",Jobcube
4,Data Scientist job in Aurora CO,Jobcube


$$
\begin{aligned}
\begin{array}{|l|l|}
\hline
\textbf{title_html} & \text{Ofertas de empleo para Data Science}\\
\hline
\textbf{body_html} & \text{Nombre del portal empleo}\\
\hline
\end{array}
\end{aligned}
$$

2. Utilizar la librería **geonamescache** para obtener una lista **cities** con el nombre de todas las ciudades del mundo  

In [93]:
gc = GeonamesCache()
cities = [city['name'] for city in gc.get_cities().values()]

* Inspeccionar el título df.title[2]

In [101]:
text_title=df.title[2]
text_title

'Data Science Instructor job in San Antonio TX '

* Crear una función que haga una comparación entre la lista **cities** y **text_title**

In [102]:
def get_city(text):
    for ele in cities:
        regex = re.compile(fr'\b{ele}\b')
        if regex.search(text):
            return ele
    return None

* Evaluar **text_title0** en **get_city**

In [104]:
matchCity=get_city(text_title)
matchCity

'San Antonio'

Se puede ver que existe una coincidencia entre **cities** y **text_title** la cual es **San Antonio**

3. Extrer coincidencias entre **cities** y **df.title** y guardalas en una lista al que llamaremos **match_cities**

In [59]:
match_cities=[]
for i in range(0, df.shape[0]):
    text=df.title[i]
    match_cities.append(get_city(text))

* Imprimir los diez primeros elementos de la lista

In [92]:
match_cities[0:10]

['York',
 'Seattle',
 'San Antonio',
 'Boston',
 'Aurora',
 'Suitland',
 'Augusta',
 'Federal',
 None,
 None]

4. Obtener dataframe cuya columna contenga los nombres de las ciudades extraidas de la columna **df['title']**

In [105]:
data = {'Titles': df.title.tolist(),'Cities': match_cities}
df_cities = pd.DataFrame(data)

In [106]:
df_cities

,Titles,Cities
0,Data Science Manager job in New York NY,York
1,Director Data Science job in Seattle WA,Seattle
2,Data Science Instructor job in San Antonio TX,San Antonio
3,"Senior Manager, Sales Data Science and Analyti...",Boston
4,Data Scientist job in Aurora CO,Aurora
...,...,...
864,Junior and Mid level Data Scientists Bristol,Bristol
865,Data Science Consultant Tadworth,None
866,Senior Financial Data Scientist- Global Hedge ...,London
867,Senior / Lead Data Scientist City of London,London


5. eliminar los elementos None

In [109]:
df_cities['Cities'].isnull().sum()

0

In [110]:
df_cities.dropna(inplace=True)
df_cities['Cities'].isnull().sum()

0

3. Exporta el nuevo data frame

In [111]:
df_cities.to_csv('jobs_cities.csv',index=False)